In [1]:
import open3d.core as o3c
import numpy as np

capacity = 10
device = o3c.Device('cpu:0')

# Hash map
A hash map is a data structure that seeks to map sparse keys (e.g.unbounded indices, strings, coordinates) from the key set K to values from the value set V with amortized O(1) access. The map is unordered.

A hash map in Open3D supports multi-dimensional coordinates as keys, and the combination of arbitrary types as values. By default we support up to 6D coordinates in integer (due to precision issue, floating point is not recommended to be regarded as keys). For higher dimensions, you may modify the macros and compile from source in [this file](https://github.com/isl-org/Open3D/blob/master/cpp/open3d/core/hashmap/Dispatch.h#L34). Both CPU and CUDA are supported -- the CUDA hash map is optimized upon [stdgpu](https://github.com/stotko/stdgpu), while the CPU version is less optimized upon [tbb's concurrent_unordered_map](https://github.com/oneapi-src/oneTBB).

## A simple example
We can first create a simple hash map from integers to integers:

In [2]:
hashmap = o3c.HashMap(capacity, 
                      key_dtype=o3c.int64, 
                      key_element_shape=(1), 
                      value_dtype=o3c.int64, 
                      value_element_shape=(1), 
                      device=device)

### Insertion
Insertion is a tensor-based operation. Two tensors of the same length can be inserted, with the same dtype and element shape as initialized in the hash map.

In [3]:
keys = o3c.Tensor([100, 200, 400, 800, 300, 200, 100], dtype=o3c.int64, device=device)
vals = o3c.Tensor([1, 2, 4, 8, 3, 2, 1], dtype=o3c.int64, device=device)
buf_indices, masks = hashmap.insert(keys, vals)

Here, `masks` indicate whether the element is successfully inserted. Duplicates will be skipped, the same as in general hash maps. **Be notified that since the insertion runs in parallel, we cannot guarantee which one is regarded as the duplicate** (e.g., input key at index 0 or index -1). 

Using advanced indexing, we can obtain which keys are actually inserted:

In [4]:
print('masks:', masks)
print('inserted keys:', keys[masks])

masks: [True True True True True False False]
Tensor[shape={7}, stride={1}, Bool, CPU:0, 0x55e6fe452110]
inserted keys: [100 200 400 800 300]
Tensor[shape={5}, stride={1}, Int64, CPU:0, 0x55e6fe4521e0]


Then we detail the usage of `buf_indices`. In our hash map, keys and values are stored in contiguous buffer tensors that could be conveniently accessed by indices. Instead of returning iterators that are less friendly to vectorized programming, we return such buffer indices. **Note these indices are not necesserily the same as input indices due to concurrency.** Also, the indices are by default stored in int32 due to the underlying implementations. **A conversion to int64 is required for advanced indexing**.

In [5]:
buf_keys = hashmap.key_tensor()
buf_vals = hashmap.value_tensor()
buf_indices = buf_indices[masks].to(o3c.int64)
print('buffer indices:', buf_indices)

inserted_keys = buf_keys[buf_indices]
inserted_vals = buf_vals[buf_indices]
print('inserted keys:', inserted_keys)
print('inserted values:', inserted_vals)

buffer indices: [0 1 3 4 2]
Tensor[shape={5}, stride={1}, Int64, CPU:0, 0x55e6fd550640]
inserted keys: [[100],
 [200],
 [400],
 [800],
 [300]]
Tensor[shape={5, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fd550fe0]
inserted values: [[1],
 [2],
 [4],
 [8],
 [3]]
Tensor[shape={5, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fe4521e0]


### Query
The query operation follows the similar convention. Note as the operation is read-only, duplicate keys are allowed and will be returned properly.

In [6]:
query_keys = o3c.Tensor([1000, 100, 300, 200, 100, 0], dtype=o3c.int64, device=device)
buf_indices, masks = hashmap.find(query_keys)
valid_keys = query_keys[masks]
buf_indices = buf_indices[masks].to(o3c.int64)
valid_vals = hashmap.value_tensor()[buf_indices]
print('found valid keys:', valid_keys)
print('found valid values:', valid_vals)

found valid keys: [100 300 200 100]
Tensor[shape={4}, stride={1}, Int64, CPU:0, 0x55e6fe453730]
found valid values: [[1],
 [3],
 [2],
 [1]]
Tensor[shape={4, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fd550e90]


### Active entries in the hash map
Sometimes we are interested in all the active entries. This can be achieved by:

In [7]:
def print_active_entries(hashmap):
    active_buf_indices = hashmap.active_buf_indices().to(o3c.int64)

    active_keys = hashmap.key_tensor()[active_buf_indices]
    print('all active keys', active_keys)

    active_vals = hashmap.value_tensor()[active_buf_indices]
    print('all active values:', active_vals)

Again, due to concurrency, the order is not guaranteed, but the key-value correspondence will be of course preserved.

### Erase
We can similarly erase keys. The behavior is similar to insert:

In [8]:
erase_keys = o3c.Tensor([100, 1000, 100], dtype=o3c.int64, device=device)
masks = hashmap.erase(erase_keys)
print('erase masks:', masks)
print('erased keys:', erase_keys[masks])

erase masks: [True False False]
Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x55e6fe44c980]
erased keys: [100]
Tensor[shape={1}, stride={1}, Int64, CPU:0, 0x55e6fe45c060]


Now we can see that active entries have been changed:

In [9]:
print_active_entries(hashmap)

all active keys [[300],
 [200],
 [400],
 [800]]
Tensor[shape={4, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fe45a760]
all active values: [[3],
 [2],
 [4],
 [8]]
Tensor[shape={4, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fd550800]


### Activate
In some cases, we know a key is occupied, but do not know the associated value -- we prefer to compute and modify it in-place afterward. This can be achieved by a chain of operations:

In [10]:
activate_keys = o3c.Tensor([1000, 0], dtype=o3c.int64, device=device)
buf_indices, masks = hashmap.activate(activate_keys)

buf_vals = hashmap.value_tensor()
# Note the assigned tensor has to be strictly in the shape of (N, 1) due to broadcasting
buf_vals[buf_indices[masks].to(o3c.int64)] = o3c.Tensor([[10], [0]], dtype=o3c.int64, device=device)
print_active_entries(hashmap)

all active keys [[300],
 [1000],
 [200],
 [400],
 [0],
 [800]]
Tensor[shape={6, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fe4532f0]
all active values: [[3],
 [10],
 [2],
 [4],
 [0],
 [8]]
Tensor[shape={6, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fd550750]


### Rehashing and reserve
Rehashing will be automatically triggered when the initial capacity is exceeded after multiple insertions, where the capacity of the hash map is doubled. Rehashing will change the location (i.e. buffer indices) of the inserted key value pairs, so an update of the buffer indices in the downstream applications is required.

In [11]:
print('size:', hashmap.size())
print('capacity:', hashmap.capacity())

keys = o3c.Tensor([700, 1200, 1500], dtype=o3c.int64, device=device)
vals = o3c.Tensor([7, 12, -1], dtype=o3c.int64, device=device)
buf_indices, masks = hashmap.insert(keys, vals)
print('size:', hashmap.size())
print('capacity:', hashmap.capacity())
print_active_entries(hashmap)

keys = o3c.Tensor([1600, 1700, 1800], dtype=o3c.int64, device=device)
vals = o3c.Tensor([16, 17, 18], dtype=o3c.int64, device=device)
buf_indices, masks = hashmap.insert(keys, vals)
print('size:', hashmap.size())
print('capacity:', hashmap.capacity())
print_active_entries(hashmap)

size: 6
capacity: 10
size: 9
capacity: 10
all active keys [[300],
 [1500],
 [700],
 [1000],
 [200],
 [400],
 [1200],
 [0],
 [800]]
Tensor[shape={9, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fe45d1e0]
all active values: [[3],
 [-1],
 [7],
 [10],
 [2],
 [4],
 [12],
 [0],
 [8]]
Tensor[shape={9, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fe463d60]
size: 12
capacity: 20
all active keys [[1700],
 [300],
 [1500],
 [700],
 [1000],
 [200],
 [1800],
 [400],
 [1200],
 [1600],
 [0],
 [800]]
Tensor[shape={12, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fe45f1c0]
all active values: [[17],
 [3],
 [-1],
 [7],
 [10],
 [2],
 [18],
 [4],
 [12],
 [16],
 [0],
 [8]]
Tensor[shape={12, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fe460960]


Rehashing is slow, as it increase the hash map capacity, collects all the active entries, and insert them back to the hash map. If we know the capacity before hand, we can pre-allocate a chunk of memory and avoid rehashing:

In [12]:
hashmap.reserve(100)
print('size:', hashmap.size())
print('capacity:', hashmap.capacity())

12
100


## Multi-valued hash map

In real-world applications, we want to map coordinates to complex data structures, e.g. a voxel position to its color and weight. This can be achieved by a multi-valued hash map. **Note this is not a multimap, and does not allow duplicate keys.** A multi-valued hash map can be constructed by

In [13]:
mhashmap = o3c.HashMap(capacity, 
                       key_dtype=o3c.int32, 
                       key_element_shape=(3), 
                       value_dtypes=(o3c.uint8, o3c.float32), 
                       value_element_shapes=(3, 1), 
                       device=device)
voxel_coords = o3c.Tensor([[0, 1, 0], [-1, 2, 3], [3, 4, 1]], dtype=o3c.int32, device=device)
voxel_colors = o3c.Tensor([[0, 255, 0], [255, 255, 0], [255, 0, 0]], dtype=o3c.uint8, device=device)
voxel_weights = o3c.Tensor([[0.9], [0.1], [0.3]], dtype=o3c.float32, device=device)
mhashmap.insert(voxel_coords, (voxel_colors, voxel_weights))

([2 1 0]
 Tensor[shape={3}, stride={1}, Int32, CPU:0, 0x55e6fe45e620],
 [True True True]
 Tensor[shape={3}, stride={1}, Bool, CPU:0, 0x55e6fe462960])

We can then query and access by indices with a slightly different routine:

In [14]:
query_coords = o3c.Tensor([[0, 1, 0]], dtype=o3c.int32, device=device)
buf_indices, masks = mhashmap.find(query_coords)

valid_keys = query_coords[masks]
buf_indices = buf_indices[masks].to(o3c.int64)
valid_colors = mhashmap.value_tensor(0)[buf_indices]
valid_weights = mhashmap.value_tensor(1)[buf_indices]
print('found coordinates:', valid_keys)
print('found colors:', valid_colors)
print('found weights:', valid_weights)

[[0 1 0]]
Tensor[shape={1, 3}, stride={3, 1}, Int32, CPU:0, 0x55e6fe46c480]
[[0 255 0]]
Tensor[shape={1, 3}, stride={3, 1}, UInt8, CPU:0, 0x55e6fe45b1e0]
[[0.9]]
Tensor[shape={1, 1}, stride={1, 1}, Float32, CPU:0, 0x55e6fe46c460]


In [17]:
def print_active_multivalue_entries(mhashmap):
    active_buf_indices = mhashmap.active_buf_indices().to(o3c.int64)

    active_keys = mhashmap.key_tensor()[active_buf_indices]
    print('all active keys:', active_keys)

    n_buffers = len(mhashmap.value_tensors())
    for i in range(n_buffers):
        active_val_i = mhashmap.value_tensor(i)[active_buf_indices]
        print('active value {}:'.format(i), active_val_i)
print_active_multivalue_entries(mhashmap)

all active keys: [[0 1 0],
 [3 4 1],
 [-1 2 3]]
Tensor[shape={3, 3}, stride={3, 1}, Int32, CPU:0, 0x55e6fe45e050]
active value 0: [[0 255 0],
 [255 0 0],
 [255 255 0]]
Tensor[shape={3, 3}, stride={3, 1}, UInt8, CPU:0, 0x55e6fe46c880]
active value 1: [[0.9],
 [0.3],
 [0.1]]
Tensor[shape={3, 1}, stride={1, 1}, Float32, CPU:0, 0x55e6fe46c520]


## Hash set
Hash set is a simplified hash map, where we do not care about the values. It preserves most of the operations in a hash map, and could be useful for removing duplicates.

In [16]:
hashset = o3c.HashSet(capacity, key_dtype=o3c.int64, key_element_shape=(1), device=device)
keys = o3c.Tensor([1, 3, 5, 7, 5, 3, 1], dtype=o3c.int64, device=device)
hashset.insert(keys)

keys = o3c.Tensor([5, 7, 9, 11], dtype=o3c.int64, device=device)
hashset.insert(keys)

def print_active_keys(hashset):
    active_buf_indices = hashset.active_buf_indices().to(o3c.int64)    
    active_keys = hashset.key_tensor()[active_buf_indices]
    print('active keys:', active_keys)

print_active_keys(hashset)

[[5],
 [9],
 [1],
 [3],
 [11],
 [7]]
Tensor[shape={6, 1}, stride={1, 1}, Int64, CPU:0, 0x55e6fe4532f0]
